In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math 
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
bucket_name = 'ir-bucket-1' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix = 'wiki_files/')
for b in blobs:
    if b.name != 'wiki_files/graphframes.sh' and b.name != 'wiki_files/':
        paths.append(full_path+b.name)


In [ ]:
nltk.download('stopwords')
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords]
    return list_of_tokens

In [ ]:
parquetFile = spark.read.parquet(*paths)

# Count number of wiki pages
N_docs = parquetFile.count()

In [ ]:
import math 
from collections import defaultdict

def doc_id_normalize(tokenize_text):
    d = defaultdict(int)
    for token in list(tokenize_text):
        d[token] += 1
    n = 0
    for token in d.keys():
        n += d[token]**2   
    if n == 0:
        return n
    return 1/math.sqrt(n)

In [ ]:
doc_text_pairs = parquetFile.select("id", "text").rdd

n_doc_text_pairs = doc_text_pairs.mapValues(tokenize).mapValues(doc_id_normalize)

normalize_doc_dict= dict(n_doc_text_pairs.collect())

In [ ]:
import pickle

name = "normalize_doc_dict"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(normalize_doc_dict, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"normalze_dict/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")